In [ ]:
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}

pd.options.mode.chained_assignment = None  # default='warn'?


data_key = pd.read_csv('key.csv')
data_key = data_key[data_key['station_nbr'] != 5]
data_weather = pd.read_csv('weather.csv')
data_weather = data_weather[data_weather['station_nbr'] != 5] ## Station 5번 제거한 나머지
data_train = pd.read_csv('train.csv')

In [ ]:
df = pd.merge(data_weather, data_key)
station_nbr = df['station_nbr']
df.drop('station_nbr', axis=1, inplace=True)
df['station_nbr'] = station_nbr
df = pd.merge(df, data_train)
df['snowfall'][df['snowfall'] == '  T'] = 0.05
df['preciptotal'][df['preciptotal'] == '  T'] = 0.005
df1 = df[df['station_nbr'] == 1]

In [3]:
df1.drop(columns = ['depart', 'codesum', 'sunrise', 'sunset', 'snowfall', 
                          'station_nbr', 'store_nbr'], inplace=True)

In [4]:
# 주말과 주중 구분 작업 # 
df1['date'] = pd.to_datetime(df['date'])
df1['week7'] = df1['date'].dt.dayofweek
df1['weekend'] = 0
df1.loc[df1['week7'] == 5, 'weekend'] = 1
df1.loc[df1['week7'] == 6, 'weekend'] = 1

In [5]:
# 형 변환 to Float
df1 = df1.apply(pd.to_numeric, errors='coerce')

# 상대 습도 추가 #
df1['relative_humility'] = 100*(np.exp((17.625*((df1['dewpoint']-32)/1.8))/(243.04+((df1['dewpoint']-32)/1.8)))/np.exp((17.625*((df1['tavg']-32)/1.8))/(243.04+((df1['tavg']-32)/1.8))))

# 체감온도 계산
df1["windchill"] = 35.74 + 0.6215*df1["tavg"] - 35.75*(df1["avgspeed"]**0.16) + 0.4275*df1["tavg"]*(df1["avgspeed"]**0.16)

# Date Drop
df1 = df1.drop(columns = 'date')

In [14]:
df1.columns

Index(['tmax', 'tmin', 'tavg', 'dewpoint', 'wetbulb', 'heat', 'cool',
       'preciptotal', 'stnpressure', 'sealevel', 'resultspeed', 'resultdir',
       'avgspeed', 'item_nbr', 'units', 'week7', 'weekend',
       'relative_humility', 'windchill'],
      dtype='object')

In [64]:
# 전처리 완료함. 따라서, 진행 시작
model = sm.OLS.from_formula("np.log1p(units) ~ tmax + tmin + tavg + dewpoint + wetbulb + heat + cool + preciptotal + stnpressure + \
sealevel + resultspeed + resultdir + avgspeed + C(item_nbr) + C(week7) + C(weekend) + relative_humility + windchill + 0", data = df1)

In [65]:
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:      np.log(units + 1)   R-squared:                       0.788
Model:                            OLS   Adj. R-squared:                  0.788
Method:                 Least Squares   F-statistic:                     2894.
Date:                Sun, 01 Jul 2018   Prob (F-statistic):               0.00
Time:                        17:49:42   Log-Likelihood:                 31405.
No. Observations:              101121   AIC:                        -6.255e+04
Df Residuals:                  100990   BIC:                        -6.130e+04
Df Model:                         130                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
C(item_nbr)[1]       -0.0590      0.00

In [50]:
# 전처리 완료함. 따라서, 진행 시작
model1 = sm.OLS.from_formula("scale(np.log(units)) ~ tmax + tmin + tavg + dewpoint + wetbulb + heat + cool + preciptotal + stnpressure + \
sealevel + resultspeed + resultdir + avgspeed + C(item_nbr) + C(week7) + C(weekend) + relative_humility + windchill + 0", data = df1)
result1 = model1.fit()
print(result1.summary())

                               OLS Regression Results                               
Dep. Variable:     scale(np.log(units + 1))   R-squared:                       0.788
Model:                                  OLS   Adj. R-squared:                  0.788
Method:                       Least Squares   F-statistic:                     2894.
Date:                      Sun, 01 Jul 2018   Prob (F-statistic):               0.00
Time:                              17:34:25   Log-Likelihood:                -64938.
No. Observations:                    101121   AIC:                         1.301e+05
Df Residuals:                        100990   BIC:                         1.314e+05
Df Model:                               130                                         
Covariance Type:                  nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------

In [27]:
# 전처리 완료함. 따라서, 진행 시작
model3 = sm.OLS.from_formula("units ~ scale(tmax)+ scale(tmin)", data = df1)

ValueError: zero-size array to reduction operation maximum which has no identity

In [68]:
df['tmax'][df['tmax'].isnull()]

Series([], Name: tmax, dtype: object)

In [25]:
df1['tmax'].dtypes

dtype('float64')